<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="100" align="center"></p>

# Semántica distribuida 

## Ingeniería de Características 

### Maestría en Ciencia de Datos
### Universidad de Sonora

#### Julio Waissman Vilanova (julio.waissman@unison.mx)



En esta libreta vamos a ver como se generan, donde se pueden descargar y algunas aplicaciones los vectores de incrustaciones de palabra (*word embeddings vectors*). En esta libreta vamos a revisar como generar un modelo de vectores de palabras con las dos técnicas más populares. En segundo lugar vamos a revisar como utilizar modelos pre-entrenados, para después ver algunas de las propiedades que parecen *mágicas* en este tipo de modelos. Por último vamos a revisar como una técnica que, a pesar de ser muy simple, es un método muy efectivo para la codificación densa de sentencias o documentos relativamente pequeños.

## 5.1. Generando vectores de palabras con `gensim`.

Para esta libreta vamos a utilizar la biblioteca de modelado de tópicos `gensim` de manera intensiva, ya que es la que proporciona las mejores herramientas para el uso de los *word embeddings* (vetores de incrustaciones) o *word vectors* (WV), y que permite utilizar modelos entrenados por otras personas y/o organizaciones. 

Vamos a generar un modelo de vectores de palabras a partir del texto del Quijote. Este procedimiento se puede aplicar con *corpus* mas grandes, para obtener mejores resultados, pero los tiempos de procesamiento no son aceptables para este curso. Se deja como tarea la obtención de dichos vectores.

Primero vamos a cargar el texto, normalizarlo y tokenizarlo. La idea es tener una lista de documentos, los cuales a su vez son una lista de palabras. Para esto vamos a utilizar los métodos que ya hemos desarrollado anteriormente. Como en todo método de PLN, el paso de normalización de texto determina fuertemente la calidad del modelo obtenido.

In [1]:
import warnings; warnings.simplefilter('ignore')

In [2]:
import re
import nltk

nltk.download('punkt')
palabras_vacias = set(nltk.corpus.stopwords.words('spanish'))

def normaliza_texto(texto):
    texto = texto.lower()
    
    # Elimina simbolos
    texto = texto.replace('\n', ' ')

    # Elimina palabras de paro
    texto = ' '.join([palabra for palabra in texto.split() 
                     if palabra not in palabras_vacias])
    
    return [nltk.tokenize.regexp_tokenize(frase, r'\w+') 
            for frase in nltk.tokenize.sent_tokenize(texto, language='spanish')]


archivo = "quijote.txt"
with open(archivo, 'r', encoding='utf8') as fp:
    texto = fp.read()
documentos = normaliza_texto(texto)

[' '.join(doc) for doc in documentos[:10]]

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juliowaissman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['produced by an anonymous project gutenberg volunteer',
 'text file corrections and new html file by joaquin cuenca abela',
 'ingenioso hidalgo don quijote mancha tasa yo juan gallo andrada escribano cámara rey señor residen consejo certifico doy fe que visto señores dél libro intitulado ingenioso hidalgo mancha compuesto miguel cervantes saavedra tasaron cada pliego dicho libro tres maravedís medio ochenta tres pliegos dicho precio monta dicho libro docientos noventa maravedís medio vender papel dieron licencia precio pueda vender mandaron tasa ponga principio dicho libro pueda vender ella',
 'y dello conste di presente valladolid veinte días mes deciembre mil seiscientos cuatro años',
 'juan gallo andrada',
 'testimonio erratas libro cosa digna corresponda original testimonio haber correcto di fee',
 'colegio madre dios teólogos universidad alcalá primero diciembre 1604 años',
 'licenciado francisco murcia llana',
 'rey cuanto parte vos miguel cervantes fecha relación habíades compu

Una vez que se obtuvo el corpus, vamos a obtener dos modelos de vectores de palabras diferentes, uno usando el método  desarrollado por *google* ([*word2vec*](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)) y otro por el método desarrollado por *facebook* ([*fastText*](https://arxiv.org/pdf/1607.04606.pdf)). 

In [3]:
from gensim import models 

modelo_w2v = models.Word2Vec(
    sentences=documentos,  #  Documentos para el entrenamiento
    size=50,              #  tamaño del vector
    window=5,              #  tamaño de la ventana del contexto
    min_count=5,           #  minima frecuencia para considerar una palabra en el vocabulario
    workers=3,             #  número de hilos para multiproceso
    sg=1                   #  si 0 usa CBOW, si 1 Skipgrama
)

modelo_fst = models.FastText(
    sentences=documentos,  #  Documentos para el entrenamiento
    size=50,              #  tamaño del vector
    window=5,              #  tamaño de la ventana del contexto
    min_count=5,           #  minima frecuencia para considerar una palabra en el vocabulario
    workers=3,             #  número de hilos para multiproceso
    sg=1                   #  si 0 usa CBOW, si 1 Skipgrama
)

Debido a que el *corpus* utilizado es relativamente pequeño, utilizamos un WV de dimensión relativamente pequeño. Lo normal es utilizar un vector entre 200 y 500 dimensiones. Por ejemplo, todos los vectores preentrenados que vamos a utilidar tienen un tamaño de 300, lo que parece ser un tamaño correcto para el español.

En este caso no es necesario, pero para un modelo con un corpus mayor, el cual puede durar horas o días de entrenamiento, es muy importante poder guardar el modelo. Cabe aclarar que estos modelos pueden ser *reentrenados* con nuevos documentos.

In [4]:
modelo_w2v.save("w2v_quijote.model")
modelo_fst.save("fst_quijote.model")

Ahora hagamos algunos experimentos con los modelos entrenados

In [10]:
def revisa_palabra(modelo_wv, palabra):
    print("\n\n¿Se encuentra la palabra {} en los wv? {}".format(palabra, palabra in modelo_wv.vocab))

    palabras_similares = modelo_wv.most_similar(positive=palabra, topn=10)
    print("\nLas 10 palabras más similares a {}".format(palabra))
    print("{:15}{:10}".format('Palabras', 'Similaridad'))
    for (palabra, simil) in palabras_similares:
        print("{:15}{}".format(palabra, simil))    

print("El vector para {} es:\n{}".format('sancho', modelo_w2v.wv['dulcinea']))
revisa_palabra(modelo_w2v.wv, 'sancho')


El vector para sancho es:
[-0.682042   -0.11337928  0.04276247 -0.08784457  0.45985296 -0.07334444
  0.59711075  0.03574967 -0.10500211 -0.44597787 -0.24065326  0.5852025
 -0.5562984   0.07006148  0.26494363 -0.30671713  0.31881428 -0.31411237
 -0.6376206   0.2914049  -0.17523001  0.12345905 -0.48658967  0.27198035
 -0.6885352  -0.7898836  -0.13964517  0.59897256 -0.77285695  0.36688456
 -0.05733135 -0.6991999   0.4437699   0.5344459   0.07712153  0.68088144
 -0.36450726 -0.11688838 -0.4273972   0.7892216  -0.3977667   0.5812498
  0.30475897  0.41383228 -0.36571032  0.02241731 -0.12783104  0.10134134
  0.04398928 -0.06070365]


¿Se encuentra la palabra sancho en los wv? True

Las 10 palabras más similares a sancho
Palabras       Similaridad
sí             0.8836077451705933
sobrina        0.8704570531845093
ama            0.8677687644958496
a              0.865344762802124
sazón          0.8650684356689453
en             0.8642257452011108
paje           0.8631842136383057
canónigo    

Veamos que pasa si tratamos de usar una palabra que no tengamos en el vocabulario 

In [11]:
revisa_palabra(modelo_w2v.wv, 'dulcineea')



¿Se encuentra la palabra dulcineea en los wv? False


KeyError: "word 'dulcineea' not in vocabulary"

Ahora hagamos lo mismo con el modelo de *fastText*

In [14]:
revisa_palabra(modelo_fst.wv, 'rey')



¿Se encuentra la palabra rey en los wv? True

Las 10 palabras más similares a rey
Palabras       Similaridad
mercader       0.9795984029769897
pide           0.9777413010597229
daño           0.977670431137085
premiar        0.9776082634925842
servirle       0.9774187207221985
indigno        0.9773309826850891
ofreciere      0.9768964052200317
enemigo        0.9765725135803223
engaño         0.9765241742134094
melindre       0.9764567017555237


In [15]:
revisa_palabra(modelo_fst.wv, 'dulcineea')



¿Se encuentra la palabra dulcineea en los wv? False

Las 10 palabras más similares a dulcineea
Palabras       Similaridad
dulcinea       0.9986686110496521
dulce          0.9635171294212341
toboso         0.9556972980499268
fermosa        0.9475977420806885
hermosa        0.9413021206855774
ora            0.93011075258255
fermosura      0.9278861880302429
señora         0.9275573492050171
princesa       0.9226405620574951
hermosura      0.9208940267562866


Y esta es la mayor cualidad del método de *fastText*, que al formarse los vectores de palabra a partir de información de $n$--gramas de letras, es capaz de extrapolar. No solo es útil para palabras mal escritas, si no tambien para palabras completamente fuera del vocabulario. Por supuesto su capacidad de abstraccion depende del contexto. 

Por ejemplo, si utilizamos una palabra en un contexto que no exista en el *corpus*, encuentra un vector de palabra, pero no significa nada en su contexto.

In [20]:
revisa_palabra(modelo_w2v.wv, 'carro')



¿Se encuentra la palabra carro en los wv? True

Las 10 palabras más similares a carro
Palabras       Similaridad
abrió          0.99595046043396
asimismo       0.9938889145851135
abajo          0.993690013885498
iban           0.9936505556106567
encima         0.9932233691215515
hacia          0.9900524616241455
ruido          0.9897677898406982
dentro         0.9895105361938477
salieron       0.9874076843261719
adarga         0.9871741533279419


## 5.2. Importar modelos existentes

Entrenar un modelo a partir de un *corpus* grande es una tarea que toma un tiempo de computo importante. Una opción común es es descargar y usar un modelo ya entrenado por alguien más (dandole el crédito correspondiente). 

Existen dos maneras de obtener los modelos: el archivo binario con el modelo completo, y la otra es el uso de los llamados [*keyed vectors*](https://radimrehurek.com/gensim/models/keyedvectors.html#why-use-keyedvectors-instead-of-a-full-model). Los modelos con llaves son archivos de texto (típicamente con terminacón `.vec`) en los que en cada renglón se encuentra la palabra, seguida por los valores de su vector de inserción, separados por tabulaciones. El tamaño de los archivos es sensiblemente menos y por lo tanto se utilizan bastante. Además, es posible limitar el número de palabras a utilizar (por ejemplo, las primeras 100,000).  En su contra, tienen que no se puede utilizar el modelo completo (y generalizar a cualquier palabra como en *FastText*. En esta libreta importaremos solamente este tipo de representación.

Los modelos entrenados para español disponibles son:

1. [**Spanish Billion Word Corpus and Embeddings**](https://crscardellino.github.io/SBWCE/). El [autor](https://crscardellino.github.io/) compilo y normalizó una gran cantidad de texto. En su página se puede descargar el modelo entrenado con el método `word2vec`. Si se utiliza este modelo (u otro modelo entrenado con este *corpus* es necesario citarlos como *Cristian Cardellino: Spanish Billion Words Corpus and Embeddings (March 2016), https://crscardellino.github.io/SBWCE/*.

2. [**Spanish Word Embeddings**](https://github.com/uchile-nlp/spanish-word-embeddings). Página del grupo de PLN de la Universidad de Chile. Presenta una lista de modelos, aunque algunos (como el de *Facebook*) se encuentran desactualizados. En la página vienen dos modelos entrenados con el *corpus* SBWC (ver inciso anterior) utilizando los métodos de *GloVe* y *fastText* entrenados por [Jorge Pérez](https://users.dcc.uchile.cl/~jperez/) de la Universidad de Chile.

3. [**fastText word vectors for 157 languages**](https://fasttext.cc/docs/en/crawl-vectors.html). *Facebook* pone a disposición de manera abierta modelos entrenados principalmente con los datos de *wikipedia* en 157 idiomas diferentes. Es posible descargar únicamente el modelo en forma de *keyed vectors* o el modelo binario.

Los cuatro modelos se encuentran previamente descargados en la imágen de Docker del grupo por lo que no es necesario hacerlo. Vamos a cargar alguno de los modelos para utilizarlos

In [21]:
from gensim.models.keyedvectors import KeyedVectors

# Ejecuta esta linea solo si no tienes ya descomprimido el modelo
# porque se puede perder bastante tiempo :-)
!gunzip -k fasttext-sbwc.vec.gz
archivo_sbwc_fst = "fasttext-sbwc.vec"

# Estos modelos los tienes que descargar tu mismo,
# si lo hacía quedaba muy grande el proyecto
# escogí el modelo FastText entrenado con el corpus SBWC solamente
# porque era el archivo de menor tamaño.

# archivo_wiki_fst = "modelos/fasttext-fcb-es.vec"
# archivo_sbwc_glo = "modelos/glove-sbwc.vec"
# archivo_sbwc_w2v = "modelos/word2vec-sbwc.vec"

num_paabras = 100000
modelo_wv = KeyedVectors.load_word2vec_format(archivo_sbwc_fst, limit=num_paabras)

Y veamos algunos ejemplos

In [22]:
revisa_palabra(modelo_wv, 'dulcinea')
revisa_palabra(modelo_wv, 'avión')



¿Se encuentra la palabra dulcinea en los wv? True

Las 10 palabras más similares a dulcinea
Palabras       Similaridad
quijote        0.7173032164573669
toboso         0.6822168827056885
celestina      0.6399698257446289
hermosura      0.6170874834060669
anarda         0.5951821804046631
bernarda       0.586138129234314
lope           0.5819821357727051
amada          0.5781254768371582
doncella       0.5770990252494812
rocinante      0.570771336555481


¿Se encuentra la palabra avión en los wv? True

Las 10 palabras más similares a avión
Palabras       Similaridad
aeronave       0.817049503326416
vuelo          0.7988545298576355
helicóptero    0.7648181915283203
aeroplano      0.7620928287506104
aviones        0.7609811425209045
boeing         0.7602846026420593
bimotor        0.7541469931602478
avioneta       0.7206918001174927
hidroavión     0.7185807824134827
volaba         0.7099997997283936


## 5.3. La ecuación *rey* - *hombre* + *mujer* $\approx$ *reina* y otras analogías

Una característica interesante de los vectores de palabras, es que estos capturan muchos tipos de similaridades, los cuales se pueden obtener a partir de operaciones aritméticas entre vectores. Por ejemplo, definamos la similaridad entre dos vectores $u$ y $v$ a través de la *similiaridad coseno* definida como:

$$
sim(u, v) = \frac{u \cdot v}{\|u\| \|v\|}
$$

que es equivalente al calculo del coseno del angulo entre ambos vectores. 

Supongamos que ahora tenemos un par de vectores $a$, $a^*$, donde $a$ es el vector de una palabra y $a*$ el de otra que tiene cierta relación con $a$. Es interesante que el vector resultante de la diferencia entre ambas palabras $a - a^*$ codifica (en algunos casos únicamente) un tipo de relación entre ambas que puede extrapolarse a otras palabras. 

Si tenemos otra palabra *positiva* $b$, debería existir una palabra $b^*$ que codifique la misma relación. Esto se puede encontrar encontrando $b^*$ tal que la similaridad entre diferencias sea máxima, esto es:

$$
b^* = \arg \max_{b^*\in V}(\cos(a - a^*, b - b^*))
$$

y esto se calcula como

$$
b^* = \arg \max_{b^*\in V}(\cos(b^*, b - a + a^*))
$$

donde $a^*$ y $b$ se conocen como vectores *positivos* y $a$ como vector *negativo* (por sus operaciones aritméticas).

Veamos algunas de esas reaciones


In [23]:
modelo_wv.most_similar(positive=['rey', 'mujer'], negative=['hombre'], topn=5)

[('reina', 0.6306586265563965),
 ('infanta', 0.5454355478286743),
 ('princesa', 0.5346059799194336),
 ('berenguela', 0.5296740531921387),
 ('consorte', 0.5245280861854553)]

In [24]:
modelo_wv.most_similar(positive=['comiendo', 'jugar'], negative=['comer'], topn=5)

[('jugando', 0.8573390245437622),
 ('disputando', 0.6486820578575134),
 ('jugado', 0.6417645215988159),
 ('entrenando', 0.6204931735992432),
 ('juegue', 0.6159106492996216)]

In [25]:
modelo_wv.most_similar(positive=['hermosillo', 'españa'], negative=['méxico'], topn=5)

[('coruña', 0.5375283360481262),
 ('almería', 0.5369367599487305),
 ('hermosilla', 0.5348933935165405),
 ('alcobendas', 0.528655469417572),
 ('murcia', 0.5281211137771606)]

In [26]:
modelo_wv.most_similar(positive=['tacos', 'colombia'], negative=['méxico'], topn=5)

[('chicharrones', 0.5050054788589478),
 ('bollos', 0.5026459693908691),
 ('chicharrón', 0.5011547207832336),
 ('empanadas', 0.4948349893093109),
 ('arepas', 0.49367496371269226)]

In [27]:
modelo_wv.most_similar(positive=['oaxaca', 'argentina'], negative=['mexico'], topn=5)

[('jujuy', 0.5951390862464905),
 ('bonaerense', 0.5945647358894348),
 ('catamarca', 0.5902442932128906),
 ('tucumán', 0.5663372278213501),
 ('mendocina', 0.5507368445396423)]

Sin embargo hay que tener mucho cuidado con estas relaciones ya que no hay ninguna base para pensar que siempre funcionan. En general con singular/plural, genero, paises/ciudades y conjugaciones funciona relativamente bien.

Tambien hay funciones para otros tipo de análisis de similaridad. Veamos unos ejemplos.

In [31]:
modelo_wv.doesnt_match(['lápiz','cuaderno','helicóptero','libros', 'sacapuntas', 'marcador'])

'helicóptero'

In [32]:
modelo_wv.closer_than('policía', 'ladrón')

['autoridades',
 'civil',
 'fuerzas',
 'ejército',
 'funcionarios',
 'agentes',
 'judicial',
 'juez',
 'agencia',
 'detención',
 'soldados',
 'víctima',
 'detenidos',
 'funcionario',
 'detenido',
 'guardia',
 'policías',
 'tránsito',
 'judiciales',
 'fiscalía',
 'agente',
 'armadas',
 'comandante',
 'operativo',
 'efectivos',
 'policial',
 'ministerial',
 'procuraduría',
 'delincuentes',
 'bomberos',
 'policiales',
 'manifestantes',
 'presuntos',
 'presunto',
 'corporación',
 'preventiva',
 'criminal',
 'criminales',
 'infantería',
 'cuartel',
 'comando',
 'brigada',
 'arresto',
 'detuvo',
 'guardias',
 'sospechosos',
 'batallón',
 'detenciones',
 'aduanas',
 'sospechoso',
 'detenidas',
 'prisiones',
 'disturbios',
 'homicidios',
 'investiga',
 'penitenciario',
 'fronteriza',
 'patrulla',
 'comisaría',
 'arrestado',
 'inspector',
 'detenida',
 'balazos',
 'delincuente',
 'forense',
 'detuvieron',
 'ladrones',
 'vialidad',
 'jefatura',
 'patrullas',
 'ministeriales',
 'funcionaria',
 'f

In [39]:
modelo_wv.most_similar_to_given('mexicali', ['lima', 'córdoba', 'managua', 'rosario', 'medellín'])

'medellín'

## 5.4. Vectores de palabras para codificar parágrafos

Algo que es interesante es como representar un documento (o párrafo, o frase) a partir de un modelo de palabras. De esa manera podríamos tener una codificación densa de cada documento en un número preestablecido de dimensiones. Entre los diferentes métodos que se han probado, uno que es bastante simple y efectivo (tomando en cuenta desconocimiento del contexto) para documentos pequeños (como entradas de *twitter*), es el de tomar los vectores de palabras de cada una de las palabras de la frase (normalizadas y sin palabras de paro) y sacar el promedio de dichos vectores. Esto nos da un vector de las mismas dimensiones que el vector de cada una de las palabras.

Y, así, este vector extrae características de cada palabra, tomando en cuenta el contexto, y utilizando un espacio de representación de una dimensión sensiblemente menor que la que tiene la representación BOW o TF-IDF.

In [43]:
import numpy as np

def frase_a_vec(frase, modelo_wv):
    N = 0
    acc = np.zeros(modelo_wv.vector_size)
    for palabra in frase:
        if palabra in modelo_wv.vocab:
            acc += modelo_wv[palabra]
            N += 1
    return acc if N==0 else acc/N

def sentencias_a_vec(frases, modelo_wv):
    vectores = np.zeros((len(frases), modelo_wv.vector_size))
    for (i, frase) in enumerate(frases):
        vectores[i,:] = frase_a_vec(frase, modelo_wv)
    return vectores
    

Vamos a probar con algunas frases

In [44]:
frases = """ 
Aunque nadie puede volver atrás y hacer un nuevo comienzo, cualquiera puede comenzar a partir de ahora y crear un nuevo final.
Mira en el nuevo día como otro regalo especial de su creador, otra oportunidad de oro.
La felicidad no puede ser ganada, es la experiencia espiritual de vida de cada minuto con amor, gracia y gratitud.
Date cuenta que tú eres quien va a llegar a donde quiere ir, nadie más.
Si tú no construyes tu sueño, alguien va a contratarte para que le ayudes a construir el suyo. 
Desearía poder hablarte, desearía poder sonreírte, desearía poder abrazarte, pero sobre todo desearía poder besarte.
Tus ojos son como dos lunas, y yo siempre quise viajar a la luna, ahora entiendo mi deseo de ser astronauta.
Unos ojos que jamás me cansaré de mirar, unos labios que siempre querré besar, pero lo mejor de todo, un corazón que jamás dejaré de amar.
Eres un lindo paisaje con el que me quiero deleitar, tus ojos las flores, tu rostro la pradera y tu boca el mar.
Nunca antes se había visto un amor igual al que siento por ti; no cabe en mi corazón, ni tampoco en este universo.
Lo único de que me arrepiento en esta vida es que no soy otro.
Siempre recuerda que tú eres absolutamente único y especial, tal como todos los demás. 
Un famoso estudio afirma que de cada 10 millones de personas que escuchan la radio, 5 millones son la mitad.
Sé que existe un mundo mucho mejor donde se cumplen todos los deseos, pero es un mundo muy caro.
Lo importante en la vida no es tener el privilegio de saber, si no de tener Google como página de inicio en tu movil.
"""

sentencias = normaliza_texto(frases)
vectores_frases = sentencias_a_vec(sentencias, modelo_wv)

# Ahora vamos a ver la similaridad de una frase resecto otras
# frase = "Siento que te debo la canción más bonita del mundo, el dibujo más bello, el poema, así como frases de amor hermosas"
# frase = "Si amas a algo, dejalo libre, si regresa es tuyo y si no, núnca lo fué"
frase = "Mi billetera es como una cebolla, abrirla me hace llorar."

frase_tok = nltk.tokenize.regexp_tokenize(frase.lower(), r'\w+')
v_frase = frase_a_vec(frase_tok, modelo_wv)

sim_cos = modelo_wv.cosine_similarities(v_frase, vectores_frases)

print("Las frases, ordenadas por su similitud con a la frase\n<<{}>>\n".format(' '.join(frase_tok)))
for i in sim_cos.argsort()[-1::-1]:
    print("({:4.3})  {}".format(sim_cos[i], ' '.join(sentencias[i])))

Las frases, ordenadas por su similitud con a la frase
<<mi billetera es como una cebolla abrirla me hace llorar>>

(0.776)  ojos dos lunas siempre quise viajar luna ahora entiendo deseo ser astronauta
(0.762)  nunca visto amor igual siento ti cabe corazón tampoco universo
(0.755)  ojos jamás cansaré mirar labios siempre querré besar mejor todo corazón jamás dejaré amar
(0.738)  lindo paisaje quiero deleitar ojos flores rostro pradera boca mar
(0.728)  aunque nadie puede volver atrás hacer nuevo comienzo cualquiera puede comenzar partir ahora crear nuevo final
(0.725)  date cuenta va llegar quiere ir nadie más
(0.724)  si construyes sueño alguien va contratarte ayudes construir suyo
(0.713)  importante vida tener privilegio saber si tener google página inicio movil
(0.701)  felicidad puede ser ganada experiencia espiritual vida cada minuto amor gracia gratitud
(0.69)  mira nuevo día regalo especial creador oportunidad oro
(0.672)  único arrepiento vida otro
(0.66)  siempre recuerda abso